In [89]:
text='''%sf -> pz, gj
%zh -> bc, st
%hk -> bc
&bc -> mn, zl, xb, mm, dh, hv, gz
%st -> bc, mm
%gv -> xf, qq
%hv -> xb
%nd -> gj, tr
%zx -> bx, ms
%sc -> ks, gj
%gr -> hn
%pl -> qq, rh
%qc -> sf, gj
%xr -> sc, gj
%zl -> zh
&gj -> ks, ld, sg, xr
%dg -> ll, bx
%nf -> bc, tg
%lz -> cv, qq
%nq -> dg, bx
%rh -> qq, lp
%xf -> qq, qj
%ms -> bx, xh
%mn -> bc, hv
&jm -> rx
%xh -> vt, bx
%pz -> gj
%vq -> bt
%gz -> nf
%bt -> gr
&sg -> jm
%fr -> bx, tb
&lm -> jm
%ld -> cl
%cv -> vq
%cl -> gj, jf
%tr -> gj, sz
%sz -> gj, ld
%dx -> hk, bc
%lr -> bx, fr
%vt -> lr, bx
%ll -> zx
broadcaster -> pl, xr, mn, xc
%lp -> lz
%mm -> gz
&qq -> lm, gr, cv, vq, lp, pl, bt
%xb -> zl
&bx -> ll, xc, db
%tb -> bx
%hn -> gv, qq
%jf -> qc, gj
%qj -> qq
%xc -> bx, pm
%tg -> bc, dx
&dh -> jm
%ks -> nd
&db -> jm
%pm -> bx, nq'''

In [69]:
text='''broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output'''

In [90]:
from collections import defaultdict
from dataclasses import dataclass


@dataclass()
class Module:
    name: str
    type: str
    inputs: list
    outputs: list
    flip_state: bool
    conjunction_memory: defaultdict
    def __init__(self, name, m_type):
        self.name = name
        self.type = m_type
        self.inputs = []
        self.outputs = []
        self.flip_state = False
        self.conjunction_memory = defaultdict(bool)
        dot.node(name, shape=shape_table[m_type])
    def __repr__(self):
        return self.name
    def pulse(self, value, sender):
        if self.type == 'broadcaster':
            for output in self.outputs:
                q.put((output, value, self.name))
        elif self.type == 'flipflop':
            if not value:
                self.flip_state = not self.flip_state
                for output in self.outputs:
                    q.put((output, self.flip_state, self.name))
        elif self.type == 'conjunction':
            self.conjunction_memory[sender] = value
            output_value = not all(self.conjunction_memory[input_module.name] for input_module in self.inputs)
            for output in self.outputs:
                q.put((output, output_value, self.name))

In [91]:
import graphviz
from graphviz import dot
from queue import Queue

module_table = {}
q = Queue()
dot = graphviz.Digraph(format='png')
shape_table = {'broadcaster': 'star', 'flipflop': 'box', 'conjunction': 'ellipse'}

for line in text.splitlines():
    left, _ = line.strip().split(' -> ')
    if left.startswith('%'):
        module_table[left[1:]] = Module(left[1:], 'flipflop')
    elif left.startswith('&'):
        module_table[left[1:]] = Module(left[1:], 'conjunction')
    else:
        module_table[left] = Module('broadcaster', 'broadcaster')

for line in text.splitlines():
    left, right = line.strip().split(' -> ')
    m = module_table['broadcaster'] if left == 'broadcaster' else module_table[left[1:]]
    for output in right.split(', '):
        if output not in module_table:
            module_table[output] = Module(output, 'broadcaster')
        dot.edge(m.name, output)
        output_module = module_table[output]
        m.outputs.append(output_module)
        output_module.inputs.append(m)

In [92]:
broadcaster = module_table['broadcaster']
low_pulses = 0
high_pulses = 0
def press_button(count):
    global low_pulses, high_pulses
    q.put((broadcaster, False, None))
    while not q.empty():
        module, pulse_value, sender = q.get()
        if pulse_value:
            high_pulses += 1
        else:
            low_pulses += 1
        # if module.name in ['xt', 'mk', 'zc', 'fp'] and not pulse_value:
        #     print(module.name, count, int(not module.flip_state))
        module.pulse(pulse_value, sender)
    return False

In [93]:
# part 1
for i in range(1000):
    press_button(i)
print(low_pulses * high_pulses)

841763884


In [94]:
print(module_table)

{'sf': sf, 'zh': zh, 'hk': hk, 'bc': bc, 'st': st, 'gv': gv, 'hv': hv, 'nd': nd, 'zx': zx, 'sc': sc, 'gr': gr, 'pl': pl, 'qc': qc, 'xr': xr, 'zl': zl, 'gj': gj, 'dg': dg, 'nf': nf, 'lz': lz, 'nq': nq, 'rh': rh, 'xf': xf, 'ms': ms, 'mn': mn, 'jm': jm, 'xh': xh, 'pz': pz, 'vq': vq, 'gz': gz, 'bt': bt, 'sg': sg, 'fr': fr, 'lm': lm, 'ld': ld, 'cv': cv, 'cl': cl, 'tr': tr, 'sz': sz, 'dx': dx, 'lr': lr, 'vt': vt, 'll': ll, 'broadcaster': broadcaster, 'lp': lp, 'mm': mm, 'qq': qq, 'xb': xb, 'bx': bx, 'tb': tb, 'hn': hn, 'jf': jf, 'qj': qj, 'xc': xc, 'tg': tg, 'dh': dh, 'ks': ks, 'db': db, 'pm': pm, 'rx': rx}
